In [2]:
# LangChain components to use
import os
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

from datasets import load_dataset
from dotenv import load_dotenv


In [3]:
load_dotenv()

ASTRA_DB_APPLICATION_TOKEN = os.environ.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.environ.get("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_NAMESPACE = os.environ.get("ASTRA_DB_NAMESPACE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [4]:
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    embedding=embedding,
    namespace=ASTRA_DB_NAMESPACE,
    collection_name="test",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
)

In [5]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

Generating train split: 100%|██████████| 450/450 [00:00<00:00, 15997.94 examples/s]

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [6]:
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Add a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [7]:
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [8]:
results = vstore.similarity_search("Our life is what we make of it", k=3)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* We are what we are because we have been what we have been. [{'author': 'freud', 'history': 'y'}]
* We become what we contemplate. [{'author': 'plato', 'knowledge': 'y', 'ethics': 'y'}]
* In the blessings as well as in the ills of life, less depends upon what befalls us than upon the way in which it is met. [{'author': 'schopenhauer', 'knowledge': 'y', 'ethics': 'y'}]
